In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [12]:
data = [[0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [1, 1, 0]]

data2 = [[0, 0, 0],
         [0, 1, 1],
         [1, 0, 1],
         [1, 1, 0]]

data3 = [[2, 3, 0],
         [4, 5, 1],
         [6, 7, 1],
         [0, 0, 2],
         [0, 0, 2],
         [0, 0, 3],
         [0, 0, 3]]

rows = list(range(1,5))
gates = list(range(1,8))

nor_tt  = pd.DataFrame(data, columns=['A', 'B', 'OUT'], index=rows)
req_tt  = pd.DataFrame(data2, columns=['A', 'B', 'OUT'], index=rows)
max_net = pd.DataFrame(data3, columns=['GATEinA', 'GATEinB', 'GATEout'], index=gates)

In [44]:
model = gp.Model('Logical Design')

# add vars
s = model.addVars(gates,
                  vtype=GRB.BINARY,
                  name='s')
tA = model.addVars(gates,
                   vtype=GRB.BINARY,
                   name='tiA')
tB = model.addVars(gates,
                   vtype=GRB.BINARY,
                   name='tiB')
x = model.addVars(gates, rows,
                  vtype=GRB.BINARY,
                  name='x')

s[1].lb = 1             # at least gate 1 must exists
x[1, 1].ub = 0
x[1, 2].lb = 1              # set the desired outputs
x[1, 3].lb = 1              # of gate 1, as the problem
x[1, 4].ub = 0              # requires

# obj function
model.setObjective(gp.quicksum(s))

# add constraints
model.addConstrs((s[i] - tA[i] >= 0 for i in gates),
                 name='if_exists_then_inputA_poss')

model.addConstrs((s[i] - tB[i] >= 0 for i in gates),
                 name='if_exists_then_inputB_poss')

model.addConstrs((s[i] - x[i, r] >= 0 for i in gates for r in rows),
                 name='if_output_then_exists')

model.addConstrs((s[max_net['GATEinA'][i]] + s[max_net['GATEinB'][i]] + tA[i] + tB[i] <= 2 for i in gates[:3]),
                 name='total_inputs')

# logic constraints
for i in gates[:3]:
    model.addConstrs(x[max_net['GATEinA'][i], r] + x[i, r] <= 1 for r in rows)
    model.addConstrs(x[max_net['GATEinB'][i], r] + x[i, r] <= 1 for r in rows)
    
for i in gates:
    model.addConstrs(nor_tt['A'][r]*tA[i] + x[i, r] <= 1 for r in rows)
    model.addConstrs(nor_tt['B'][r]*tB[i] + x[i, r] <= 1 for r in rows)
    
for i in gates[3:]:
    model.addConstrs(nor_tt['A'][r]*tA[i] 
                     + nor_tt['B'][r]*tB[i] 
                     + x[i, r] 
                     - s[i] >= 0 for r in rows)
    
for i in gates[:3]:
    model.addConstrs(nor_tt['A'][r]*tA[i] 
                     + nor_tt['B'][r]*tB[i] 
                     + x[max_net['GATEinA'][i], r]
                     + x[max_net['GATEinB'][i], r]
                     + x[i, r] 
                     - s[i] >= 0 for r in rows)

In [45]:
model.write('Logical Design.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 153 rows, 49 columns and 336 nonzeros
Model fingerprint: 0x1ef526a6
Variable types: 0 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 5.0000000
Presolve removed 153 rows and 49 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%


In [50]:
print('Existent gates')
for i in gates:
    if s[i].x > 0:
        print('Gate {}'.format(i))
        print('\tInputs A = {}, and B = {}'.format(tA[i].x, tB[i].x))

Existent gates
Gate 1
	Inputs A = 0.0, and B = 0.0
Gate 2
	Inputs A = 1.0, and B = 1.0
Gate 3
	Inputs A = 0.0, and B = 0.0
Gate 6
	Inputs A = 0.0, and B = 1.0
Gate 7
	Inputs A = 1.0, and B = 0.0
